## Simulations
1.Figure 1: seed=1, $\varphi=0.01, \lambda=0.02, I=100, J=2, T=1000$

2.Figure 2: seed=1, $\varphi=0.1, \lambda=0.5, I=100, J=2, T=1000$

3.Figure 3: seed=1, $\varphi=0.01, \lambda=0.02, I=10, J=2, T=1000$

4.Figure 4: seed=1, $\varphi=0.01, \lambda=0.02, I=100, J=2, T=1000$, but $c_0=c_1$

4.Figure 5: seed=1, $\varphi=0.01, \lambda=0.02, I=100, J=2, T=1000$, but $c_0=c_1$ (zoomed version of Figure 5)

5.Figure 6: seed=1, $\varphi=0.01, \lambda=0.02, I=100, J=15, T=1000$

5.Figure 7: seed=1, $\varphi=0.01, \lambda=0.02, I=100, J=15, T=1000$ (zoomed version of Figure 6)

6.Figure 8: seed=1, $\varphi=0.01, \lambda=0.02, I=100, J=15, T=1000$, but $c_0=c_1=...=c_{14}$

7.Figure 9: seed=17, $\varphi=0.08, \lambda=0.1, I=80, J=10, T=1000$

7.Figure 10: seed=17, $\varphi=0.08, \lambda=0.1, I=80, J=10, T=1000$ (zoomed version of Figure 9)

8.Figure 11: seed=5, $\varphi=0.05, \lambda=0.02, I=150, J=10, T=1000$

9.Figure 12: seed=17, $\varphi=0.1, \lambda=0.5, I=100, J=2, T=1000$, but the second box should be run twice

In [ ]:
import numpy as np
import seaborn as sns 
np.random.seed(1)


I=100
J=2
T=1000


phi=0.01
lambd=0.02

def optim_x(i,j,t, q_hat, q_leftt):
    x_hat=b[i]/p[j,t]-sum(x[i, : ,t]) 
    x_bar=np.minimum(np.maximum(x_hat,0),(q_leftt[j,t])) 
    q_l=q_leftt[j,t]-x_bar
    return x_bar, q_l
   

def optim_q_hat(p, costs1,j,t):
    q_hat=(p[j,t])/(2*costs1[j,0])
    return np.maximum(q_hat,0)


def next_p(j,t):
    if q_leftt[j, t-1] ==0: 
        pn = p[j, t-1] + phi                
    else:
        pn= p[j, t-1] - lambd * q_leftt[j,t-1] 
    return(pn)




x=np.zeros((I,J,T)) #quantity bought by buyers 
x_bar=np.zeros((I,J)) ################################################ 
x_hat=np.zeros((I,J,T)) ############################################## 
p=np.zeros((J,T+1)) #prices
q_hat=np.zeros((J,T)) #quantity produced
q_leftt=np.zeros((J,T)) #quantity left
profit=np.zeros((J,T)) #profit of each producer at time t
capital=np.zeros((J,T+1)) #capital of each producer
bankruptcy_hist =np.zeros((J,T))
bucy=0 #number of bankrupt firms
mon_left=np.zeros((I,T)) #the money that remains with the buyer after the ерisоde 


cost_coef = np.zeros((J,2)) #coefficients for quadratic function
theta_coef = np.zeros ((J,1)) #probably useless, coefficients of keeping costs
for j in range(J):
    cost_coef[j,] =  np.random.uniform(low=0, high=1, size=2) 
    theta_coef[j,] = np.random.rand(1)
    #cost_coef[j,]=cost_coef[0,]     ###command to check equal coefficients 

b=np.zeros((I,1)) #coefficient for buyers utility
for i in range(I):
    b[i,] = np.random.rand(1)


In [ ]:
buyer_idx = list(range(I)) #list for shuffle command 

p[:,0] = np.random.uniform(0,1,J) #initial prices
for t in range(T):
    firm_idx = np.argsort(p[:,t])
    for j in firm_idx:
        q_hat[j,t]=optim_q_hat(p,cost_coef,j,t)
        
        q_leftt[j,t]=q_hat[j,t]+q_leftt[j,t-1] #definition of q_leftt
        
        
        np.random.shuffle(buyer_idx)
        for i in buyer_idx:
            x[i,j,t], q_leftt[j,t]=optim_x(i,j,t, q_hat, q_leftt) 
            
        
    
        profit[j,t]=p[j,t]*sum(x[ : ,j,t])-cost_coef[j,0]*q_hat[j,t]**2-theta_coef[j]*q_leftt[j,t]
            

        capital[j,t]=capital[j,t-1]+profit[j,t]
            
        p[j,t+1]=next_p(j,t+1)
            
        if capital[j,t]<0: #update of the coefficients (one firm went bankrupt, new one has appeared)
                cost_coef[j,]= np.random.uniform(low=0, high=1, size=2)
                capital[j,t]=0
                bucy+=1
                bankruptcy_hist[j,t]=1
                p[j,t+1]=np.random.rand()
        
            


In [ ]:
#%pip install scipy
from scipy.optimize import minimize

def objective(p, b, cost_coef):
    return (sum(b[:]/p) - sum(p/(2*cost_coef[:,0])))**2

p_initial_guess = 1.0

p_bounds = (0, None)

result = minimize(objective, p_initial_guess, args=(b, cost_coef), bounds=[p_bounds])
if result.success:
    optimal_p = result.x[0]
    minimum_value = result.fun
    print("Mininmum of the function is 0, p=", optimal_p)
else:
    print("Minimum of the function doesnt equal to 0, could be error, p=", optimal_p)

zho=np.zeros(J)
for j in range(J):
    print(f'Mean of the prices for the Seller {j} (from 500 episode)=',np.mean(p[j,500:]))
    zho[j]=np.mean(p[j,500:])
print('Mean of the average prices=',np.mean(zho))


In [ ]:
import matplotlib.pyplot as plt
eq_price=[optimal_p]*T
z=range(T)
for j in range(J):
    sns.lineplot(x=z, y=p[j,z], label=f'Seller {j}')
    
plt.plot(z, eq_price, 'r--', label='Equilibrium price')



# Добавление названий осей
plt.xlabel('Time')
plt.ylabel('Prices')
plt.title(f'# of epsisodes = {T}, # of sellers = {J}, # of buyers = {I} ')

# Создание легенды
#plt.legend()

# Отображение графика
#plt.show()

In [ ]:
print(bankruptcy_hist)
print(bucy)

if np.any(bankruptcy_hist == 1):

    indices = np.where(bankruptcy_hist == 1)
    row_index = indices[0][0]
    col_index = indices[1][0]
    
    print("1 was found in matrix -> somebody went bankrupt")
else:
    print("1 was not found in matrix -> nobody went bankrupt")



In [ ]:
z=range(500,601)
eq_price=[optimal_p]*len(z)
for j in range(J):
    sns.lineplot(x=z, y=p[j,z], label=f'Seller {j}')
plt.plot(z, eq_price, 'r--', label='Equilibrium price')
plt.xlabel('Time')
plt.ylabel('Prices')
plt.title(f'# of epsiodes = ({z[0]}:{z[len(z)-1]}), # of sellers = {J}, # of buyers = {I} ')

In [ ]:
#Checking the cost coefficients
import pandas as pd 
dp=pd.DataFrame(cost_coef[:,0])
print(dp)

In [ ]:
#Who has the smallest and highest cost coefficient
print('min сoef' ,np.min(cost_coef[:,0]))  
print('(min) for the seller', np.argmin(cost_coef[:,0]))
print('max coef=' ,np.max(cost_coef[:,0]))  
print('(max) for the seller', np.argmax(cost_coef[:,0]))  
print(np.mean(cost_coef[:,0]), cost_coef[np.argmin(var_vec),0])


In [ ]:
#Average profit analyses starting from episode 500
mean_prof=np.zeros(J)
for j in range(J):
    #print(f'Mean q_hat of seller {j}=', np.mean(q_hat[j,500:]))        #average quantity produced also can be checked
    mean_prof[j]=np.mean(profit[j,500:])
    print(f'Average profit of firm {j}=',np.mean(profit[j,500:]))
print('Overall mean is',np.mean(mean_prof))

In [ ]:
import pandas as pd 
df=pd.DataFrame(p)
df2=pd.DataFrame(bankruptcy_hist)
print(df2)

In [ ]:
#Price variance analyses 
var_vec=np.zeros(J)
for j in range(J):
    var_vec[j]=np.var(p[j,500:1000])
    print(f'Variance of prices of seller {j} = ', np.var(p[j,500:1000]))
print('min var=' ,np.min(var_vec))  
print('(min) for the seller', np.argmin(var_vec))
print('max var=' ,np.max(var_vec))  
print('(max) for the seller', np.argmax(var_vec))    

In [ ]:
#Quantity sold analyses
z=range(500,601)
eq_price=[optimal_p]*len(z)
for j in range(J):
    sns.lineplot(x=z, y=sum(x[:,j,z]), label=f'Seller {j}')
